In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
count = 0
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        count += 1
        if count >= 10:
            break
    if count >= 10:
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import cv2
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight

In [ ]:

labels = ['PNEUMONIA', 'NORMAL']

def get_training_data(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))
                img_arr = cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB)
                resized_arr = cv2.resize(img_arr, (150, 150))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data, dtype=object)

In [ ]:
train = get_training_data('/kaggle/input/chest-xray-pneumonia/chest_xray/train')
test = get_training_data('/kaggle/input/chest-xray-pneumonia/chest_xray/test')

In [ ]:
x_train = []
y_train = []
for feature, label in train:
    x_train.append(feature)
    y_train.append(label)

x_test = []
y_test = []
for feature, label in test:
    x_test.append(feature)
    y_test.append(label)

In [ ]:
x_train = np.array(x_train) /255
x_test = np.array(x_test) /255
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

In [ ]:
data_generator = ImageDataGenerator(
    rotation_range=45,
    zoom_range=0.3,
    width_shift_range=0.25,
    height_shift_range=0.25,
    horizontal_flip=True,
    shear_range=0.25,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)
data_generator.fit(x_train)

In [ ]:
from keras.layers import BatchNormalization

img_size = 150
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))


base_model.trainable = True
for layer in base_model.layers[:-5]:#here we are unfreezing last 5 layers
    layer.trainable = False

inputs = Input(shape=(img_size, img_size, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid')(x)
model = Model(inputs, outputs)


model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)          │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet50 (Functional)                │ (None, 5, 5, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_6           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,850,625 (90.98 MB)

 Trainable params: 1,317,377 (5.03 MB)

 Non-trainable params: 22,533,248 (85.96 MB)

In [ ]:


learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5, min_lr=1e-7)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=7, verbose=1, restore_best_weights=True)

history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=(x_val, y_val),
    class_weight=class_weight_dict,
    callbacks=[learning_rate_reduction, early_stopping]
)

Epoch 1/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 25s 297ms/step - accuracy: 0.7038 - loss: 1.0825 - val_accuracy: 0.2567 - val_loss: 1.9547 - learning_rate: 1.0000e-04
Epoch 2/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 21s 282ms/step - accuracy: 0.7493 - loss: 0.8662 - val_accuracy: 0.8659 - val_loss: 0.6966 - learning_rate: 1.0000e-04
Epoch 3/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 21s 283ms/step - accuracy: 0.7874 - loss: 0.7718 - val_accuracy: 0.2577 - val_loss: 2.1255 - learning_rate: 1.0000e-04
Epoch 4/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 21s 283ms/step - accuracy: 0.7904 - loss: 0.7062 - val_accuracy: 0.9023 - val_loss: 0.5410 - learning_rate: 1.0000e-04
Epoch 5/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 21s 283ms/step - accuracy: 0.7940 - loss: 0.6537 - val_accuracy: 0.8927 - val_loss: 0.4913 - learning_rate: 1.0000e-04
Epoch 6/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 21s 283ms/step - accuracy: 0.8037 - loss: 0.6248 - val_accuracy: 0.7720 - val_loss: 0.6745 - learning_rate: 1.0000e-04
Epoch 7/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step - accu

In [ ]:
print("Loss of the model is - ", model.evaluate(x_test, y_test)[0])
print("Accuracy of the model is - ", model.evaluate(x_test, y_test)[1] * 100, "%")

20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 135ms/step - accuracy: 0.9194 - loss: 0.3405
Loss of the model is -  0.529679000377655
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.9194 - loss: 0.3405
Accuracy of the model is -  84.61538553237915 %


In [ ]:
predictions = (model.predict(x_test) > 0.5).astype("int32").reshape(-1)
print("THE CLASSIFICATION REPORT OF RESNET50 IS AS FOLLOWS:")
print(classification_report(y_test, predictions, target_names=['Pneumonia', 'Normal']))

20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 308ms/step
THE CLASSIFICATION REPORT OF RESNET50 IS AS FOLLOWS:
              precision    recall  f1-score   support

   Pneumonia       0.82      0.96      0.89       390
      Normal       0.91      0.65      0.76       234

    accuracy                           0.85       624
   macro avg       0.87      0.81      0.82       624
weighted avg       0.86      0.85      0.84       624



# LEARNINGS
>   So we are getting an accuracy of 85 percent from resnet which is lower than that of the CNN Model
> Slight overfitting is done but the Recall of pneumonia is great which is a good sign F1 scores are also decent but not better than CNN
> Data augmentation and probably oversampling methods could be used since dataset is imbalanced
> also Validation Set is really small only 10 images therefore we are ignoring that and dealing with the training and testing dataset only. We can improve the resnet by doing some hyperparameter tuning as well. This was a really great learning